In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!unzip "/content/drive/My Drive/perros_refinados.zip" -d "/content/"

!pip install keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import os
import math

In [ ]:
IMG_SIZE = 299  # Tamaño de las imágenes
BATCH_SIZE = 32
EPOCHS = 20
DATASET_DIR = "perros_refinados"


# 1. Generadores de datos
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2,  # 20% para validación
)

# Generador para validación (sin augmentación)
val_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Solo normalización
    validation_split=0.2,
)

# Flujos de datos
train_gen = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
)

val_gen = val_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
)

# 2. Modelo preentrenado (Transfer Learning)
base_model = InceptionV3(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights="imagenet")

# Congelar capas base
base_model.trainable = False

# 3. Añadir capas superiores
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Regularización
x = Dense(256, activation="relu")(x)
outputs = Dense(train_gen.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=outputs)

# 4. Compilación del modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

steps_per_epoch = math.ceil(train_gen.samples / BATCH_SIZE)
validation_steps = math.ceil(val_gen.samples / BATCH_SIZE)

# 5. Entrenamiento
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
)

# 6. Descongelar algunas capas para ajuste fino (Fine-tuning)
base_model.trainable = True
for layer in base_model.layers[:200]:  # Congelar las primeras capas
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.00001), loss="categorical_crossentropy", metrics=["accuracy"])

# Entrenar de nuevo con ajuste fino
fine_tune_history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=40,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
)

# 8. Evaluación final
loss, accuracy = model.evaluate(val_gen)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

In [ ]:
model.save("CanemSCANDOG.keras")